# 🥙 LSTM on Recipe Data

In this notebook, we'll walk through the steps required to train your own LSTM on the recipes dataset

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import json
import re
import string

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, losses

## 0. Parameters <a name="parameters"></a>

In [2]:
VOCAB_SIZE = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25

## 1. Load the data <a name="load"></a>

In [3]:
# Load the full dataset
with open("..\\..\\..\\data\\epirecipes\\full_format_recipes.json", encoding='utf-8',  errors='ignore') as json_data:
    recipe_data = json.load(json_data)

In [4]:
recipe_data[0]

{'directions': ['1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool.',
  '2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper.',
  '3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.'],
 'fat': 7.0,
 'date': '2006-09-01T04:00:00.000Z',
 'categories': ['Sandwich',
  'Bean',
  'Fruit',
  'Tomato',
  'turkey',

In [5]:
# Filter the dataset
filtered_data = [
    "Recipe for " + x["title"] + " | " + " ".join(x["directions"])
    for x in recipe_data
    if "title" in x
    and x["title"] is not None
    and "directions" in x
    and x["directions"] is not None
]

In [6]:
# Count the recipes
n_recipes = len(filtered_data)
print(f"{n_recipes} recipes loaded")

20111 recipes loaded


In [7]:
example = filtered_data[0]
print(example)

Recipe for Lentil, Apple, and Turkey Wrap  | 1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool. 2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper. 3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.


## 2. Tokenise the data

In [8]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]

In [9]:
# Display an example of a recipe
example_data = text_data[0]
example_data

'Recipe for Lentil , Apple , and Turkey Wrap | 1 . Place the stock , lentils , celery , carrot , thyme , and salt in a medium saucepan and bring to a boil . Reduce heat to low and simmer until the lentils are tender , about 30 minutes , depending on the lentils . ( If they begin to dry out , add water as needed . ) Remove and discard the thyme . Drain and transfer the mixture to a bowl ; let cool . 2 . Fold in the tomato , apple , lemon juice , and olive oil . Season with the pepper . 3 . To assemble a wrap , place 1 lavash sheet on a clean work surface . Spread some of the lentil mixture on the end nearest you , leaving a 1 - inch border . Top with several slices of turkey , then some of the lettuce . Roll up the lavash , slice crosswise , and serve . If using tortillas , spread the lentils in the center , top with the turkey and lettuce , and fold up the bottom , left side , and right side before rolling away from you . '

In [10]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [11]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [12]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [13]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: .
3: ,
4: and
5: to
6: in
7: the
8: with
9: a


In [14]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[  26   16 1733    3  428    3    4  221  212   27   11    2   64    7
  300    3  924    3  353    3  576    3  307    3    4   24    6    9
   29   80    4   84    5    9   69    2  153   17    5  134    4   70
   10    7  924   79   85    3   19  126   12    3 1135   28    7  924
    2   34   92  316  601    5  162  124    3   18   39  151  542    2
   35   71    4  206    7  307    2  120    4   40    7   31    5    9
   21   22   67   60    2   15    2  255    6    7  265    3  428    3
  109  104    3    4  252   37    2   63    8    7   33    2   36    2
    5 1567    9  212    3   64   11 2863  105   28    9  370  387  207
    2  166  256   14    7 1733   31   28    7  613 1908  215    3  447
    9   11   13   53  813    2   72    8  688  160   14  221    3   46
  256   14    7  711    2  263   99    7 2863    3  284  446    3    4
   68    2   92   87  704    3  166    7  924    6    7  168    3   72
    8    7  221    4  711    3    4  255   99    7  169    3 1286   96
    3 

## 3. Create the Training Set

In [15]:
# Create the training set of recipes and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

## 4. Build the LSTM <a name="build"></a>

In [16]:
inputs = layers.Input(shape=(None,), dtype="int32")
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS, return_sequences=True)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         1000000   
                                                                 
 lstm (LSTM)                 (None, None, 128)         117248    
                                                                 
 dense (Dense)               (None, None, 10000)       1290000   
                                                                 
Total params: 2,407,248
Trainable params: 2,407,248
Non-trainable params: 0
_________________________________________________________________


In [17]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    lstm = models.load_model("./models/lstm", compile=False)

## 5. Train the LSTM <a name="train"></a>

In [18]:
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam", loss_fn)

In [19]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }  # <1>

    def sample_from(self, probs, temperature):  # <2>
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]  # <3>
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:  # <4>
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)  # <5>
            sample_token, probs = self.sample_from(y[0][-1], temperature)  # <6>
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)  # <7>
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("recipe for", max_tokens=100, temperature=1.0)

In [20]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [21]:
lstm.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/25
628/629 [============================>.] - ETA: 0s - loss: 4.4237
generated text:
recipe for tarts glass steamed the covered | food set in ready from covered , large 2 mango in cover it to next hour with serve on sundaes with high , 1 and whisk . stir . chiles until chilled rice to a bowls . size , chicken , pan , 5 heat in medium do lifted and add saucepan until high and baking nonstick vibrantly tent with coat ; pepper . 

629/629 [==============================] - 18s 21ms/step - loss: 4.4224
Epoch 2/25
629/629 [==============================] - ETA: 0s - loss: 3.0769
generated text:
recipe for pepper chicken with dill | cook garlic in large bowl and cook seconds . roll potatoes into cheesecloth ( ( do not needed with kept 2 minutes and beat in a bowl , and chop . whisk 1 powder in boiling water until going with batches from 1 / 4 - quart heavy large pot over low heat until slightly , then lay pan into a pitcher in 10 minutes . place yams and let stand 5 minutes . turn ma

In [22]:
# Save the final model
lstm.save("./models/lstm")

INFO:tensorflow:Assets written to: ./models/lstm\assets


INFO:tensorflow:Assets written to: ./models/lstm\assets


## 6. Generate text using the LSTM

In [23]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [24]:
info = text_generator.generate(
    "recipe for roasted vegetables | chop 1 /", max_tokens=10, temperature=1.0
)


generated text:
recipe for roasted vegetables | chop 1 / 2 teaspoon



In [25]:
print_probs(info, vocab)


PROMPT: recipe for roasted vegetables | chop 1 /
2:   	51.73%
4:   	35.83%
3:   	9.25%
8:   	1.47%
1:   	0.28%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 2
cup:   	40.06%
teaspoon:   	19.98%
inch:   	7.95%
-:   	5.68%
tsp:   	5.42%
--------



In [26]:
info = text_generator.generate(
    "recipe for roasted vegetables | chop 1 /", max_tokens=10, temperature=0.2
)


generated text:
recipe for roasted vegetables | chop 1 / 2 cup



In [27]:
print_probs(info, vocab)


PROMPT: recipe for roasted vegetables | chop 1 /
2:   	86.23%
4:   	13.75%
3:   	0.02%
8:   	0.0%
1:   	0.0%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 2
cup:   	96.97%
teaspoon:   	2.99%
inch:   	0.03%
-:   	0.01%
tsp:   	0.0%
--------



In [28]:
info = text_generator.generate(
    "recipe for chocolate ice cream |", max_tokens=7, temperature=1.0
)
print_probs(info, vocab)


generated text:
recipe for chocolate ice cream | 1


PROMPT: recipe for chocolate ice cream |
bring:   	14.33%
in:   	13.89%
stir:   	10.48%
combine:   	7.44%
heat:   	4.37%
--------



In [29]:
info = text_generator.generate(
    "recipe for chocolate ice cream |", max_tokens=7, temperature=0.2
)
print_probs(info, vocab)


generated text:
recipe for chocolate ice cream | in


PROMPT: recipe for chocolate ice cream |
bring:   	47.39%
in:   	40.52%
stir:   	9.93%
combine:   	1.79%
heat:   	0.13%
--------



In [31]:
info = text_generator.generate(
    "recipe for roasted vegetables | ", max_tokens=100, temperature=1.0
)


generated text:
recipe for roasted vegetables |  preheat oven to 400°f . smash garlic cloves and spices in processor . add potatoes and rub around pork breast to 8 intervals to hold pork . refrigerate chicken 30 minutes . meanwhile , combine shrimp sauce , barley , radicchio and bacon in large bowl . add remaining ingredients ; toss to coat . season salad to taste with salt and pepper and mix well . spoon syrup over fish . add sauce in raisins ; season to taste with salt and pepper . ( can be made 1 day ahead . cool slightly



In [32]:
info = text_generator.generate(
    "recipe for roasted vegetables | ", max_tokens=100, temperature=0.2
)


generated text:
recipe for roasted vegetables |  preheat oven to 400°f . place 1 / 2 cup oil in heavy large pot . add 1 cup water to pan . cover and cook until tender , about 15 minutes . uncover and simmer until vegetables are tender , about 15 minutes longer . season with salt and pepper . meanwhile , cook pasta in large pot of boiling salted water until tender , about 5 minutes . drain . return to pot . add 1 cup water and bring to boil . reduce heat to medium - low and simmer until liquid

